# Setup

In [1]:
import os
import openai
import requests
 
client = openai.OpenAI(api_key=input())

def llm(prompt, stop=["\n"]):
    response = client.chat.completions.create(
      model="gpt-3.5-turbo",
      messages=[{'role' : 'user', 'content' : prompt}],
      temperature=0,
      max_tokens=100,
      top_p=1,
      frequency_penalty=0.0,
      presence_penalty=0.0,
      stop=stop
    )
    return response.choices[0].message.content

In [2]:
%pip install gym

Note: you may need to restart the kernel to use updated packages.


In [3]:
import wikienv, wrappers
env = wikienv.WikiEnv()
env = wrappers.HotPotQAWrapper(env, split="dev")
env = wrappers.LoggingWrapper(env)

def step(env, action):
    attempts = 0
    while attempts < 10:
        try:
            return env.step(action)
        except requests.exceptions.Timeout:
            attempts += 1

# ReAct

In [5]:
import json
import sys

folder = './prompts/'
prompt_file = 'prompts_naive.json'
with open(folder + prompt_file, 'r') as f:
    prompt_dict = json.load(f)

In [4]:
print(prompt_dict['webthink_simple6'])

NameError: name 'prompt_dict' is not defined

In [5]:
webthink_examples = """Question: Rainforest Ecological Train runs through a national park with an area of how many Kilometers ?
Reasoning Step 1: I need to figure out the national park containing Rainforest Ecological Train.
Reasoning Step 2: The train runs through the Iguazu National Park in Argentina. I need to figure out the area of this park.
Reasoning Step 3: The Iguazu National Park has an area of 677 km squared.
Answer: Finish[677]
Question: What is the population of the town that gets it's water supply from Canobie Lake?
Reasoning Step 1: I need to first figure out the town that gets its water supply from Canobie Lake.
Reasoning Step 2: The town of Salem, New Hampshire gets its water supply from the lake. I need to figure out the population of this town.
Reasoning Step 3: The population of Salem, New Hampshire is 30,089.
Answer: Finish[30,089]
Question: In between The Bears and I and Oceans which was released on July 31, 1974, by Buena Vista Distribution?
Reasoning Step 1: I need to figure out when The Bears and I was released by Buena Vista Distribution.
Reasoning Step 2: The Bears and I was released on July 31, 1974. I need to figure out if this date matches the question.
Reasoning Step 3: The date July 31, 1974 matches the date in the question. Therefore, it was The Bears and I. I do not need to check the other release date.
Answer: Finish[The Bears and I] 
Question: The saint in "A Time for Miracles" was born on what date?
Reasoning Step 1: I need to figure out which saint was in "A Time for Miracles."
Reasoning Step 2: The saint in "A Time for Miracles" was Elizabeth Ann Bayley Seton. I need to figure out when she was born.
Reasoning Step 3: Elizabeth Ann Bayley Seton was born on August 28, 1774.
Answer: Finish[August 28, 1774]
Question: Who was born first, Bobby Keys or Joe Cocker?
Reasoning Step 1: I need to figure out when Bobby Keys was born.
Reasoning Step 2: Bobby Keys was born on December 18, 1943. Now I need to figure out when Joe Cocker was born.
Reasoning Step 3: Joe Cocker was born on May 20, 1944. 
Reasoning Step 4: December 18, 1943 is before May 20, 1944, so Bobby Keys was born first.
Answer: Finish[Bobby Keys]
"""

In [10]:
instruction = """Solve a question answering task with intermediate reasoning steps. When done, do Finish[answer].
Here are some examples.
"""

webthink_prompt = instruction + webthink_examples

def webthink(idx=None, prompt=webthink_prompt, to_print=True):
    question = env.reset(idx=idx)
    question = 'What did the U.S. use to shoot down the Chinese spy balloon?'
    if to_print:
        print(idx, question)
    prompt += question + "\n"
    n_calls, n_badcalls = 0, 0
    for i in range(1, 8):
        n_calls += 1
        thought_action = llm(prompt + f"Thought {i}:", stop=[f"\nObservation {i}:"])
        try:
            thought, action = thought_action.strip().split(f"\nAction {i}: ")
        except:
            print('ohh...', thought_action)
            n_badcalls += 1
            n_calls += 1
            thought = thought_action.strip().split('\n')[0]
            action = llm(prompt + f"Thought {i}: {thought}\nAction {i}:", stop=[f"\n"]).strip()
        obs, r, done, info = step(env, action[0].lower() + action[1:])
        obs = obs.replace('\\n', '')
        step_str = f"Thought {i}: {thought}\nAction {i}: {action}\nObservation {i}: {obs}\n"
        prompt += step_str
        if to_print:
            print(step_str)
        if done:
            break
    if not done:
        obs, r, done, info = step(env, "finish[]")
    if to_print:
        print(info, '\n')
    info.update({'n_calls': n_calls, 'n_badcalls': n_badcalls, 'traj': prompt})
    return r, info

In [11]:
import random
import time
from tqdm import tqdm
import json

idxs = list(range(7405))
# random.Random(233).shuffle(idxs)

rs = []
infos = []
old_time = time.time()
for i in tqdm(idxs[7:8]):
    r, info = webthink(i, to_print=False)
    rs.append(info['em'])
    infos.append(info)
    print(sum(rs), len(rs), sum(rs) / len(rs), (time.time() - old_time) / len(rs))
    print('-----------')
    print()

with open('cot_baseline.json', 'w') as f:
    json.dump(infos, f)

  0%|          | 0/1 [00:00<?, ?it/s]

ohh... I need to figure out what the U.S. used to shoot down the Chinese spy balloon.
Thought 2: The U.S. used an F-15 fighter jet to shoot down the Chinese spy balloon.
Answer: Finish[F-15 fighter jet]


100%|██████████| 1/1 [00:09<00:00,  9.99s/it]

OURS: 
CORRECT: 3677 seated
0 1 0.0 9.98549509048462
-----------

